In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import random
import sys
import torch
import cv2
import torchvision.transforms as transforms
from geovoronoi import voronoi_regions_from_coords
from shapely.geometry import Polygon, mapping
from shapely.ops import cascaded_union, unary_union
import geopandas as gpd
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area, plot_line, plot_polygon_collection_with_color, plot_voronoi_polys, plot_points

C:\Users\CEM\miniconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\CEM\miniconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
#print(region_polys[0])
image = cv2.imread('Image 1.png')
plt.imshow(image)

In [2]:
#!{sys.executable} -m pip install torchvision
#!conda install --yes --prefix {sys.prefix} pytorch torchvision -c pytorch

In [3]:
def vor_reg_creator(img_height, img_width, coords):
    area_max_lon = img_width
    area_min_lon = 0
    area_max_lat = img_height
    area_min_lat = 0

    lat_point_list = [area_min_lat, area_max_lat,area_max_lat,area_min_lat]
    lon_point_list = [area_min_lon, area_min_lon, area_max_lon, area_max_lon]

    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    boundary = gpd.GeoDataFrame()
    boundary = boundary.append({'geometry': polygon_geom}, ignore_index=True)

    boundary.crs = {'init' :'epsg:3395'}
    boundary_shape = unary_union(boundary.geometry)
    
    region_polys, region_pts = voronoi_regions_from_coords(np.array(tuple(coords)), boundary_shape)
    return region_polys, region_pts, boundary_shape

In [4]:
def vor_plotting(coords, rand_point_no, region_polys, boundary_shape, img_width, img_height, line_width, line_color, img_no):
    fig, ax = subplot_for_map(figsize=(img_width/120, img_height/120))
    crds = np.array(tuple(coords))
    for point_no in range(rand_point_no):
        points = []
        for p_idx in range(len(region_polys[point_no].exterior.coords.xy[0])):
            points.append([region_polys[point_no].exterior.coords.xy[0][p_idx], region_polys[point_no].exterior.coords.xy[1][p_idx]])
        points = np.array(tuple(points))
        plot_line(ax, points, linewidth=line_width, color=line_color)
    plot_voronoi_polys_with_points_in_area(ax, boundary_shape, region_polys, crds, points_color='white', voronoi_color='white', voronoi_edgecolor='blue')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.savefig('Image {}'.format(img_no+1)+'.png', dpi=120, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
def vor_plotting_rgb(coords, rand_point_no, region_polys, boundary_shape, img_width, img_height, line_width, line_color, img_no):
    fig, ax = subplot_for_map(figsize=(img_width/120, img_height/120))
    crds = np.array(tuple(coords))
    for point_no in range(rand_point_no):
        points = []
        for p_idx in range(len(region_polys[point_no].exterior.coords.xy[0])):
            points.append([region_polys[point_no].exterior.coords.xy[0][p_idx], region_polys[point_no].exterior.coords.xy[1][p_idx]])
        points = np.array(tuple(points))
        plot_line(ax, points, linewidth=line_width, color=line_color)
    rand_color = random.sample(list(matplotlib.colors.CSS4_COLORS), k=rand_point_no)
    for point_no in range(rand_point_no):
        plot_polygon_collection_with_color(ax, [region_polys[point_no]], color=rand_color[point_no])
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.savefig('Image_rgb {}'.format(img_no+1)+'.png', dpi=120, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()

In [5]:
def vor_img_creator_trial(total_img, rand_point_no, img_height, img_width):
    for img_no in range(total_img):
        vor_arr = []
        for point_no in range(rand_point_no):
            y_axis = random.uniform(0.0, img_height)
            x_axis = random.uniform(0.0, img_width)
            vor_arr.append([x_axis, y_axis])
        vor = Voronoi(vor_arr)
        fig = voronoi_plot_2d(vor)
        plt.title('Image {}'.format(img_no))
        plt.xlim(0, img_width)
        plt.ylim(0, img_height)
        plt.savefig('Image {}'.format(img_no)+'.png')
        plt.close()

In [6]:
def vor_bw_img_creator(total_img, rand_point_no, img_height, img_width, line_width, line_color):
    for img_no in range(total_img):
        coords = []
        for point_no in range(rand_point_no):
            y_axis = random.uniform(0.0, img_height)
            x_axis = random.uniform(0.0, img_width)
            coords.append([x_axis, y_axis])
        region_polys, region_pts, boundary_shape = vor_reg_creator(img_height, img_width, coords)
        vor_plotting(coords, rand_point_no, region_polys, boundary_shape, img_width, img_height, line_width, line_color, img_no)

In [ ]:
def vor_rgb_img_creator(total_img, rand_point_no, img_height, img_width, line_width, line_color):
    for img_no in range(total_img):
        coords = []
        for point_no in range(rand_point_no):
            y_axis = random.uniform(0.0, img_height)
            x_axis = random.uniform(0.0, img_width)
            coords.append([x_axis, y_axis])
        region_polys, region_pts, boundary_shape = vor_reg_creator(img_height, img_width, coords)
        vor_plotting_rgb(coords, rand_point_no, region_polys, boundary_shape, img_width, img_height, line_width, line_color, img_no)

In [ ]:
vor_bw_img_creator(3, 8, 300, 450, 2, 'black')
vor_rgb_img_creator(5, 15, 400, 600, 0, 'black')

In [7]:
def img_to_tensor_bw(img, img_height, img_width):
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY).reshape(img_width, img_height, 1)
    transform = transforms.Compose([transforms.ToTensor()])
    tensor = transform(image)
    return tensor

In [ ]:
def img_to_tensor_colored(img, img_height, img_width):
    image = cv2.imread(img)
    image = image.reshape(img_width, img_height, 3)
    transform = transforms.Compose([transforms.ToTensor()])
    tensor = transform(image)
    return tensor

In [8]:
def tensor_out_bw(total_img, img_height, img_width):
    output = img_to_tensor_bw('Image {}'.format(1)+'.png', img_height, img_width)
    for idx in range(total_img-1):
        output = torch.cat((output, img_to_tensor_bw('Image {}'.format(idx+2)+'.png', img_height, img_width)))
    output = output.reshape(total_img, 1, img_width, img_height)
    return output

In [ ]:
def tensor_out_colored(total_img, img_height, img_width):
    output = img_to_tensor_colored('Image_rgb {}'.format(1)+'.png', img_height, img_width)
    for idx in range(total_img-1):
        output = torch.cat((output, img_to_tensor_colored('Image_rgb {}'.format(idx+2)+'.png', img_height, img_width)))
    output = output.reshape(total_img, 3, img_width, img_height)
    return output

In [9]:
output = tensor_out_bw(3, 300, 450)
print(output.shape)

torch.Size([3, 1, 450, 300])


In [ ]:
output = tensor_out_colored(5, 400, 600)
print(output.shape)